In [40]:
import sys

In [1]:
from IPython.lib import kernel
kernel.get_connection_file()


'/run/user/1000/jupyter/kernel-506a1572-7c57-4dd8-bf77-3d5511b218cb.json'

In [2]:
from elasticsearch import Elasticsearch
from pymongo import MongoClient
import pandas as pd
es = Elasticsearch()
db = MongoClient().scraper_test_dev

In [38]:
query = {
  "query": {
    "filtered": {
      "filter": {
        "bool": {
          "must": [
            {
              "script": {
                "script": "doc['type'].size() == 1"
              }
            },
            {
              "script": {
                "script": "doc['platforms'].size() == 1"
              }
            },
            {
              "term": {
                "data_source": "array-express"
              }
            },{
              "terms": {
                "type": [
#                   "transcription profiling by array",
                  "RNA-seq of coding RNA"
                ]
              }
            }
          ]
        }
      }
    }
  },
  "aggs": {
    "platforms": {
      "terms": {
        "field": "platforms",
        "size": 5000        
      },
      "aggs": {
        "types": {
          "terms": {
            "field": "type"
          }    
        }
      }
    }
  },
  "size": 0
}

res = es.search(index='series_dev', body=query)


In [5]:
len(res)

5

In [8]:
types_map = {
    "transcription profiling by array": "Expression profiling by array",
    "RNA-seq of coding RNA": "Expression profiling by high throughput sequencing"
}

In [39]:
res

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'aggregations': {'platforms': {'buckets': [{'doc_count': 1,
     'key': 'A-MEXP-2210',
     'types': {'buckets': [{'doc_count': 1, 'key': 'RNA-seq of coding RNA'}],
      'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0}}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0}},
 'hits': {'hits': [], 'max_score': 0.0, 'total': 1},
 'timed_out': False,
 'took': 784}

In [9]:
platforms_expr = [(x['key'], types_map[x['types']['buckets'][0]['key']]) for x in res['aggregations']['platforms']['buckets']]
len(platforms_expr)

1426

In [10]:
platforms_expr[:10]

[('GPL570', 'Expression profiling by array'),
 ('GPL1261', 'Expression profiling by array'),
 ('GPL198', 'Expression profiling by array'),
 ('GPL4133', 'Expression profiling by array'),
 ('GPL96', 'Expression profiling by array'),
 ('GPL6246', 'Expression profiling by array'),
 ('GPL6244', 'Expression profiling by array'),
 ('GPL6887', 'Expression profiling by array'),
 ('GPL1355', 'Expression profiling by array'),
 ('GPL4134', 'Expression profiling by array')]

In [22]:
[p for p, t in platforms_expr if t != "Expression profiling by array"][:10]

['GPL11154',
 'GPL10999',
 'GPL16791',
 'GPL9115',
 'GPL9052',
 'GPL16288',
 'GPL9442',
 'GPL13393',
 'GPL15433',
 'GPL14603']

In [11]:
from collections import defaultdict

types_platforms = defaultdict(list)
for p, t in platforms_expr:
    types_platforms[t].append(p)

In [30]:
def iter_search(es, index, query, **kvargs):
    page_size = 1000
    count = int(es.search(index=index, body=query, size=1, fields='')['hits']['total'])
    print(count)
    for page in range(0, count, page_size):
        res = es.search(index=index, body=query, from_=page, size=page_size, **kvargs)
        for hit in res['hits']['hits']:
            yield hit

def iter_bucket(it, page_size):
    res = []
    l = 1
    for x in it:
        res.append(x)
        if l % page_size == 0:
            l = 0
            yield res
            res = []
        l += 1
    
    if res:
        yield res
    
from elasticsearch.helpers import bulk
def update_by_query(es, index, query, update, bucket_size=1000):
    l = 0
    ids = (
        s['_id'] for s in iter_search(es, index, query, fields='')
    )
    
#     print(bucket_size)
    for bucket in iter_bucket(ids, bucket_size):
        actions = [
            {
                '_op_type': 'update',
                '_index': index,
                '_type': index,
                '_id': x,
                'doc': update
            }
            for x in bucket
        ]
        l += len(actions)
#         print(l)
        res = bulk(es, actions)
#         print(res)
    
    return l
        
    
    
    

In [29]:
def update_for_exptype(experimant_type, data_source, platforms=None):
    platforms = platforms or types_platforms[experiment_type]
    query = {
        "query": {
            "filtered": {
                "filter": {
                    "bool": {
                        "must": [{
                            "terms": {
                                "platform": platforms
                            }
                        }, {
                                "term": {
                                "data_source": data_source
                            }
                            }]
                    }
                }
            }
        }
    }

    update = {
        "assigned": {
            "experiment_type": experiment_type
        }
    }
    return update_by_query(es, 'samples_dev', query, update, bucket_size=1000)

## Updating samples with platforms "Expression profiling by array"

In [33]:
experiment_type = "Expression profiling by array"
update_for_exptype(experiment_type, 'array-express')

450768


450768

## Updating samples with platforms "Expression profiling by high throughput sequencing"

In [35]:
list(types_platforms.keys())

['Expression profiling by array',
 'Expression profiling by high throughput sequencing']

In [36]:
types_platforms['Expression profiling by high throughput sequencing']

[]

In [34]:
experiment_type = "Expression profiling by high throughput sequencing"
update_for_exptype(experiment_type, 'array-express')

0


0

## Checking counts


In [95]:
## Array
query = {
        "query": {
            "filtered": {
                "filter": {
                    "terms": {
                        "platform": types_platforms["Expression profiling by array"]
                    }
                }
            }
        }
    }
es.count(index='samples_dev', body=query)

{'_shards': {'failed': 0, 'successful': 5, 'total': 5}, 'count': 542701}

In [107]:
## Array (by assigned exp type)
query = {
        "query": {
            "filtered": {
                "filter": {
                    "term": {
                        "assigned.experiment_type": "Expression profiling by array"
                    }
                }
            }
        }
    }
es.count(index='samples_dev', body=query)

{'_shards': {'failed': 0, 'successful': 5, 'total': 5}, 'count': 433369}

In [106]:
query = {
        "query": {
            "filtered": {
                "filter": {
                    "terms": {
                        "platform": ['GPL570']
                    }
                }
            }
        }
    }
es.count(index='samples_dev', body=query)

{'_shards': {'failed': 0, 'successful': 5, 'total': 5}, 'count': 108902}